#### TODO sensor_sims add names
#### TODO consistnecy analysis per sensors

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob as glob
import torch

In [6]:
swat_sensors = [
    "Timestamp", "FIT101", "LIT101", "MV101", "P101", "P102", "AIT201", "AIT202",
    "AIT203", "FIT201", "MV201", "P201", "P203", "P204", "P205", "P206",
    "DPIT301", "FIT301", "LIT301", "MV301", "MV302", "MV303", "MV304",
    "P302", "AIT401", "AIT402", "FIT401", "LIT401", "P402", "P403",
    "UV401", "AIT501", "AIT502", "AIT503", "AIT504", "FIT501", "FIT502", "FIT503",
    "FIT504", "P501", "PIT501", "PIT502", "PIT503", "FIT601",
    "P602", "Normal/Attack"
]

# Dictionary with continuous index as key and [sensor_name, type, group] as value
swat_sensors_dict = {
    0: ["FIT101", "Sensor", 1],
    1: ["LIT101", "Sensor", 1],
    2: ["MV101", "Actuator", 1],
    3: ["P101", "Actuator", 1],
    4: ["P102", "Actuator", 1],
    5: ["AIT201", "Sensor", 2],
    6: ["AIT202", "Sensor", 2],
    7: ["AIT203", "Sensor", 2],
    8: ["FIT201", "Sensor", 2],
    9: ["MV201", "Actuator", 2],
    10: ["P201", "Actuator", 2],
    11: ["P203", "Actuator", 2],
    12: ["P204", "Actuator", 2],
    13: ["P205", "Actuator", 2],
    14: ["P206", "Actuator", 2],
    15: ["DPIT301", "Sensor", 3],
    16: ["FIT301", "Sensor", 3],
    17: ["LIT301", "Sensor", 3],
    18: ["MV301", "Actuator", 3],
    19: ["MV302", "Actuator", 3],
    20: ["MV303", "Actuator", 3],
    21: ["MV304", "Actuator", 3],
    22: ["P302", "Actuator", 3],
    23: ["AIT401", "Sensor", 4],
    24: ["AIT402", "Sensor", 4],
    25: ["FIT401", "Sensor", 4],
    26: ["LIT401", "Actuator", 4],
    27: ["P402", "Actuator", 4],
    28: ["P403", "Actuator", 4],
    29: ["UV401", "Actuator", 4],
    30: ["AIT501", "Sensor", 5],
    31: ["AIT502", "Sensor", 5],
    32: ["AIT503", "Sensor", 5],
    33: ["AIT504", "Sensor", 5],
    34: ["FIT501", "Sensor", 5],
    35: ["FIT502", "Sensor", 5],
    36: ["FIT503", "Sensor", 5],
    37: ["FIT504", "Sensor", 5],
    38: ["P501", "Actuator", 5],
    39: ["PIT501", "Sensor", 5],
    40: ["PIT502", "Sensor", 5],
    41: ["PIT503", "Sensor", 5],
    42: ["FIT601", "Sensor", 6],
    43: ["P602", "Actuator", 6]
}

### Data sanity check

In [7]:
def build_tensor():
    # Expand the tilde to the home directory
    pattern = os.path.expanduser(f"~/baselines/GANF/checkpoint/model/GANF_water_seed_18/graph_best*.pt")

    # Find all matching files
    file_paths = sorted(glob(pattern))
    print(f"Found {len(file_paths)} files matching the pattern")

    # Function to load a matrix from a text file
    def load_matrix(file_path):
        return torch.load(file_path)


    # Load all matrices into a tensor
    matrices = []
    for file_path in file_paths:
        print(f"Loading {file_path}...")
        matrix = load_matrix(file_path)
        matrix = matrix.cpu().numpy()
        matrices.append(matrix)
        print(f"Loaded matrix with shape {matrix.shape}")

    # Stack all matrices into a single tensor
    if matrices:
        learned_graph_tensor = np.stack(matrices)
        print(f"Created tensor with shape {learned_graph_tensor.shape}")
    else:
        print("No matrices found")
    return learned_graph_tensor
swat_tensor = build_tensor()


TypeError: 'module' object is not callable

### node influence distribution

In [ ]:
import numpy as np
from glob import glob

def analyze_node_connectivity(matrices, analysis=True):
    # Load all matching matrices
    results = []

    for matrix in matrices:
        # Calculate degree for each node
        node_degrees = np.sum(matrix, axis=1)

        # Calculate statistics
        degree_mean = np.mean(node_degrees)
        degree_std = np.std(node_degrees)
        degree_variance = np.var(node_degrees)

        # Get highest and lowest connected nodes
        most_connected = np.argmax(node_degrees)
        least_connected = np.argmin(node_degrees)

        results.append({
            'degree_mean': degree_mean,
            'degree_std': degree_std,
            'degree_variance': degree_variance,
            'most_connected': (most_connected, node_degrees[most_connected]),
            'least_connected': (least_connected, node_degrees[least_connected])
        })

        print(f"Mean node degree: {degree_mean:.4f}")
        print(f"Std of degrees: {degree_std:.4f}")
        print(f"Variance of degrees: {degree_variance:.4f}")
        print(f"Most connected node: {most_connected} (degree: {node_degrees[most_connected]:.4f})")
        print(f"Least connected node: {least_connected} (degree: {node_degrees[least_connected]:.4f})")

        # Calculate overall statistics across all matrices
    if analysis:
        means = np.array([result['degree_mean'] for result in results])
        stds = np.array([result['degree_std'] for result in results])
        variances = np.array([result['degree_variance'] for result in results])

        print("\nOverall Statistics:")
        print(f"Mean of means: {np.mean(means):.4f} ± {np.std(means):.4f}")
        print(f"Mean of std devs: {np.mean(stds):.4f} ± {np.std(stds):.4f}")
        print(f"Mean of variances: {np.mean(variances):.4f} ± {np.std(variances):.4f}")

    return results



In [ ]:
analyze_node_connectivity(swat_tensor)

Mean node degree: 1.2021
Std of degrees: 0.6502
Variance of degrees: 0.4228
Most connected node: 30 (degree: 2.9280)
Least connected node: 18 (degree: 0.0261)
Mean node degree: 0.7609
Std of degrees: 0.2864
Variance of degrees: 0.0820
Most connected node: 25 (degree: 1.3996)
Least connected node: 18 (degree: 0.1197)

Overall Statistics:
Mean of means: 0.9815 ± 0.2206
Mean of std devs: 0.4683 ± 0.1819
Mean of variances: 0.2524 ± 0.1704


[{'degree_mean': np.float32(1.2020801),
  'degree_std': np.float32(0.6502452),
  'degree_variance': np.float32(0.42281878),
  'most_connected': (np.int64(30), np.float32(2.9280012)),
  'least_connected': (np.int64(18), np.float32(0.02614277))},
 {'degree_mean': np.float32(0.76089805),
  'degree_std': np.float32(0.28642446),
  'degree_variance': np.float32(0.08203897),
  'most_connected': (np.int64(25), np.float32(1.3996139)),
  'least_connected': (np.int64(18), np.float32(0.11967802))}]

### structural convergence


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarities(matrices):
    # Calculate cosine similarities between all pairs of matrices
    num_matrices = len(matrices)
    all_similarities = []

    for i in range(num_matrices):
        for j in range(i+1, num_matrices):
            # Flatten matrices for cosine similarity calculation
            matrix_i_flat = matrices[i].reshape(1, -1)
            matrix_j_flat = matrices[j].reshape(1, -1)

            sim = cosine_similarity(matrix_i_flat, matrix_j_flat)[0, 0]
            all_similarities.append(sim)
            print(f"Cosine similarity between matrix {i} and {j}: {sim:.4f}")

    # Calculate average similarity
    avg_similarity = np.mean(all_similarities) if all_similarities else None
    print(f"Average cosine similarity: {avg_similarity:.4f}")

    # Calculate cosine similarity for each sensor (row in the matrix)
    sensor_similarities = []

    # Check if matrices have the same shape
    if all(m.shape == matrices[0].shape for m in matrices):
        num_sensors = matrices[0].shape[0]
        for sensor_idx in range(num_sensors):
            sensor_sims = []
            for i in range(num_matrices):
                for j in range(i+1, num_matrices):
                    # Get the row for this sensor from both matrices
                    sensor_i = matrices[i][sensor_idx, :].reshape(1, -1)
                    sensor_j = matrices[j][sensor_idx, :].reshape(1, -1)

                    sim = cosine_similarity(sensor_i, sensor_j)[0, 0]
                    sensor_sims.append(sim)

            # Average similarity for this sensor
            avg_sensor_sim = np.mean(sensor_sims) if sensor_sims else None
            sensor_similarities.append(avg_sensor_sim)
            print(f"Sensor {swat_sensors_dict[sensor_idx][0]} average similarity: {avg_sensor_sim:.4f}")
    else:
        print("Cannot calculate per-sensor similarities: matrices have different shapes")
    # all matrix similarities, average sim of all matrices, similarity per sensor for all matrices,
    return all_similarities, avg_similarity, sensor_similarities

In [ ]:
all_sims, avg_sim, sensor_sims = calculate_cosine_similarities(swat_tensor)


Cosine similarity between matrix 0 and 1: 0.6794
Average cosine similarity: 0.6794
Sensor 0 average similarity: 0.8354
Sensor 1 average similarity: 0.5595
Sensor 2 average similarity: 0.7810
Sensor 3 average similarity: 0.8837
Sensor 4 average similarity: 0.6121
Sensor 5 average similarity: 0.7765
Sensor 6 average similarity: 0.4251
Sensor 7 average similarity: 0.7043
Sensor 8 average similarity: 0.7888
Sensor 9 average similarity: 0.8648
Sensor 10 average similarity: 0.6383
Sensor 11 average similarity: 0.8255
Sensor 12 average similarity: 0.4446
Sensor 13 average similarity: 0.8492
Sensor 14 average similarity: 0.3963
Sensor 15 average similarity: 0.7307
Sensor 16 average similarity: 0.8192
Sensor 17 average similarity: 0.6078
Sensor 18 average similarity: 0.3906
Sensor 19 average similarity: 0.8495
Sensor 20 average similarity: 0.8508
Sensor 21 average similarity: 0.6776
Sensor 22 average similarity: 0.7595
Sensor 23 average similarity: 0.7122
Sensor 24 average similarity: 0.4741
Se

### homophily

In [ ]:
def calculate_weighted_homophily(tensor,sensor_dictionary,type=1):
    num_matrices = len(tensor)
    homophily_values = []

    for i in range(len(tensor)):
            # Load the adjacency matrix
            adj_matrix = tensor[i]

            # Ensure the matrix is symmetric (in case it's not)
            adj_matrix = (adj_matrix + adj_matrix.T) / 2

            # Initialize counters
            weighted_total = 0
            weighted_same_attribute = 0

            # Iterate through all node pairs (only upper triangle to avoid double counting)
            for i in range(len(adj_matrix)):
                attr_i = sensor_dictionary[i][type]

                for j in range(i+1, len(adj_matrix)):  # upper triangle only
                    attr_j = sensor_dictionary[j][type]

                    # Weight of the edge between nodes i and j
                    weight = adj_matrix[i, j]

                    # Add to total weighted edges
                    weighted_total += weight

                    # If attributes match, add to same-attribute counter
                    if attr_i == attr_j:
                        weighted_same_attribute += weight

            # Calculate homophily for this matrix
            homophily = weighted_same_attribute / weighted_total if weighted_total > 0 else 0
            homophily_values.append(homophily)

            print(f"Weighted homophily (type {type}): {homophily:.4f}")
            print(f"Total weighted edges: {weighted_total:.4f}")
            print(f"Same-attribute weighted edges: {weighted_same_attribute:.4f}")

    return homophily_values
calculate_weighted_homophily(swat_tensor,swat_sensors_dict,type=1)

Weighted homophily (type 1): 0.4857
Total weighted edges: 26.4457
Same-attribute weighted edges: 12.8459
Weighted homophily (type 1): 0.5010
Total weighted edges: 16.7396
Same-attribute weighted edges: 8.3871


[np.float32(0.48574436), np.float32(0.50103366)]